In [24]:
import numpy as np
import pandas as pd
import pickle
import os
import glob
import random
from sklearn.preprocessing import MultiLabelBinarizer

In [17]:
vinbig_full = pd.read_csv("./train.csv")

In [18]:
vinbig_full.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN


In [20]:
mask_imgs_original = [mask.replace("./originals/", "").replace("_json", "") for mask in glob.glob("./originals/*_json")]
len(mask_imgs_original)

95

In [34]:
train_ids = random.sample(mask_imgs_original, int(0.6 * len(mask_imgs_original)))

In [35]:
val_test_ids = set(mask_imgs_original) - set(train_ids)

In [36]:
val_ids = random.sample(val_test_ids, int(0.2 * len(mask_imgs_original)))
test_ids = list(set(val_test_ids) - set(val_ids))

/tmp/ipykernel_2034/4218299585.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  val_ids = random.sample(val_test_ids, int(0.2 * len(mask_imgs_original)))


In [38]:
len(set(train_ids + val_ids + test_ids))

95

In [41]:
# construct train/val/test files
def dump_image_sets(image_ids, binarized_label_list, set_name = "train"):
    images_f = open(f"{set_name}_binarized_list.txt", "w")
    for idx, image_id in enumerate(image_ids):
        images_f.write(f"{image_id} {' '.join(map(str, list(binarized_label_list[idx])))}" + os.linesep)
    images_f.close()

def construct_files(image_ids, vinbig_df, dataset):
    label_lists = []
    for image_id in image_ids:
        labels = set(vinbig_df.query(f"image_id=='{image_id}'")["class_name"].values)
        label_lists.append(labels)
        
    mlb = MultiLabelBinarizer(classes=['Aortic enlargement', 'Atelectasis', 'Calcification',
       'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration',
       'Lung Opacity', 'No finding', 'Nodule/Mass', 'Other lesion',
       'Pleural effusion', 'Pleural thickening', 'Pneumothorax',
       'Pulmonary fibrosis'])

    binarized_labels = mlb.fit_transform([label for label in label_lists])
    
    dump_image_sets(image_ids, binarized_labels, dataset)

In [42]:
construct_files(train_ids, vinbig_full, "train")
construct_files(val_ids, vinbig_full, "val")
construct_files(test_ids, vinbig_full, "test")